# Import & Setup

In [2]:
import sys
sys.path.append(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/My-Package")

import pandas as pd
import numpy as np
from myfunctions import clean_path
import h3

df = pd.read_csv(filepath_or_buffer=clean_path(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021.csv"),
                 index_col='Stop ID')

# H3 Encoding

* Resolution options:
  * Official list w/ area https://h3geo.org/docs/core-library/restable/
  * Visualize at https://observablehq.com/@four43/h3-index-visualizer
* Looks like resolution=(7 or 8) is the way to go

In [13]:
def convert_to_h3(dataframe:pd.DataFrame, resolution:int) -> pd.Series:
    return h3.geo_to_h3(lat=dataframe['Longitude'], 
                        lng=dataframe['Latitude'],
                        resolution=resolution)

In [20]:
df['H3 Encoding - Res=7'] = df.apply(convert_to_h3, resolution=7, axis=1)
df['H3 Encoding - Res=8'] = df.apply(convert_to_h3, resolution=8, axis=1)

# Aggregate

Get # Stops in a cell, # Accidents, # Fatalities, # 

In [27]:
df.groupby(by='H3 Encoding - Res=7').count()

,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,Personal Injury,Property Damage,...,Warning,Male,Female,Probable Cause,Arrest,DateTime,Speed Limit,Recorded Speed,H3 Encoding,H3 Encoding - Res=8
H3 Encoding - Res=7,,,,,,,,,,,,,,,,,,,,,
87754e64dffffff,1421,1421,1421,1421,1421,1421,1421,1421,1421,1421,...,1421,1421,1421,1421,1421,1421,140,30,1421,1421
87f042809ffffff,192,192,192,192,192,192,192,192,192,192,...,192,192,192,192,192,192,34,18,192,192
87f04280dffffff,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,1,0,4,4
87f042825ffffff,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,5,0,9,9
87f042828ffffff,41,41,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,8,1,41,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87f0462f6ffffff,101,101,101,101,101,101,101,101,101,101,...,101,101,101,101,101,101,31,3,101,101
87f046389ffffff,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,0,0,3,3
87f04638affffff,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,1


In [ ]:
def aggregate_by_hex(dataframe:pd.DataFrame, resolution:int):
    grouped = df.groupby(by=f'H3 Encoding - Res={resolution}')
    

In [3]:
df

,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,Belts,Personal Injury,Property Damage,...,Arrest Type,Citation,Warning,Male,Female,Probable Cause,Arrest,DateTime,Speed Limit,Recorded Speed
Stop ID,,,,,,,,,,,,,,,,,,,,,
0,MCP,"2nd District, Bethesda","LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",CORDELL ST @ NORFOLK AVE.,38.989743,-77.097770,0,0,0,0,...,A - Marked Patrol,1,0,1,0,0,0,2019-08-11 20:02:00,NaN,NaN
1,MCP,"2nd District, Bethesda",EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,NBI270 AT MIDDLEBROOK RD,39.174110,-77.246170,0,0,0,0,...,A - Marked Patrol,1,0,1,0,0,0,2019-08-12 13:41:00,55.0,85.0
2,MCP,"5th District, Germantown",DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,MIDDLEBROOK AN 355,39.182016,-77.238221,0,0,0,0,...,A - Marked Patrol,1,0,1,0,0,0,2019-08-12 21:00:00,NaN,NaN
3,MCP,"5th District, Germantown",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,GERMANTOWN RD AND ALE HOUSE,39.160508,-77.284023,0,0,0,0,...,A - Marked Patrol,1,0,1,0,0,0,2019-08-12 21:43:00,NaN,NaN
4,MCP,"2nd District, Bethesda",FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,EASTWEST/ 355,38.984247,-77.090548,0,0,0,0,...,A - Marked Patrol,1,0,1,0,0,0,2019-08-12 21:30:00,30.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13368,MCP,"6th District, Gaithersburg / Montgomery Village",EXCEEDING THE POSTED SPEED LIMIT OF 30 MPH,E VILLAGE AVE @ HARRON VALLEY WAY,39.184952,-77.176118,0,0,0,0,...,Q - Marked Laser,0,1,1,0,0,0,2020-10-20 16:17:00,30.0,NaN
13368,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVING VEHICLE IN EXCESS OF REASONABLE AND PR...,E VILLAGE AVE @ HARRON VALLEY WAY,39.184952,-77.176118,0,0,0,0,...,Q - Marked Laser,0,1,1,0,0,0,2020-10-20 16:17:00,46.0,NaN
13369,MCP,"6th District, Gaithersburg / Montgomery Village",EXCEEDING THE POSTED SPEED LIMIT OF 30 MPH,E VILLAGE AVE @ HARRON VALLEY WAY,39.185863,-77.176383,0,0,0,0,...,Q - Marked Laser,0,1,0,1,0,0,2020-10-20 16:30:00,30.0,NaN
